In [ ]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt
import string
from scipy import stats
import scipy
import statsmodels.api as sm

#visualizing results
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context("poster")
sns.set_style("ticks")
sns.set(font_scale=2)

from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as shc
from sklearn.metrics import silhouette_score, davies_bouldin_score, mean_squared_error, mean_absolute_error, r2_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split, cross_val_score, cross_val_predict, GridSearchCV, RandomizedSearchCV
from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, roc_auc_score

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as shc

from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC, LinearSVC 
from sklearn.neighbors import KNeighborsClassifier 

import shap
# load JS visualization code to notebook
shap.initjs()

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

## read in and create dfs 

In [ ]:
path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/Clinical projects/DCTE/data/1031_Peskind_20220401/merged_short.csv'

data = pd.read_csv(path)
print(data.shape)

data.head()

In [ ]:
#remove cv columns
cv_cols = [col for col in data.columns if '_cv' in col]
data = data.loc[:, ~data.columns.isin(cv_cols)]

In [ ]:
fbm_serum = ['p_Eotaxin', 'p_Eotaxin_3', 'p_IL_8', 'p_IL_8_2', 'p_IP_10', 'p_MCP_1', 'p_MCP_4', 
             'p_MDC', 'p_MIP_1alpha', 'p_MIP_1_beta', 'p_TARC',
             
             'p_INF_gamma', 'p_IL_1_beta', 'p_IL_2', 'p_IL_4', 'p_IL_6', 'p_IL_10', 
             'p_IL_12p70', 'p_IL_13', 'p_TNF_alpha',
             
             'p_GM_CSF', 'p_IL_1_alpha', 'p_IL_5', 'p_IL_7', 'p_IL_12_IL_23p40', 'p_IL_15', 
             'p_IL_16',  'p_IL_17A', 'p_TNF_beta', 'p_VEGF_A',

             'p_IL_1RA', 'p_IL_3', 'p_IL_9', 'p_IL_17A_F', 'p_IL_17B', 'p_IL_17C', 'p_IL_17D', 'p_TSLP',
             
             'p_IL_17AGenB', 'p_IL_21', 'p_IL_22', 'p_IL_23', 'p_IL_27', 'p_IL_31', 'p_MIP_3alpha',
             
             'p_bFGF', 'p_PlGF', 'p_Tie_2', 'p_VEGF_A_2', 'p_VEGF_C', 'p_VEGF_D', 'p_Fit_1',
             
             'p_CRP', 'p_ICAM_1', 'p_SAA', 'p_VCAM_1', 
             
             'p_asyn', 'Plasma_Ab40_avg', 'Plasma_Ab42_avg', 'Plasma_GFAP_avg', 'Plasma_NFL_avg',
             'Plasma_PDGFRbeta', 'pT181', 'pT231', 'Plasma_totaltau',]
    

fbm_csf = ['c_Eotaxin', 'c_Eotaxin_3', 'c_IL_8', 'c_IL_8_2',  'c_IP_10', 'c_MCP_1', 'c_MCP_4', 
           'c_MDC', 'c_MIP_1alpha', 'c_MIP_1_beta', 'c_TARC',
           
           'c_INF_gamma',  'c_IL_1_beta', 'c_IL_2', 'c_IL_4', 'c_IL_6', 'c_IL_10', 'c_IL_12p70', 
           'c_IL_13', 'c_TNF_alpha',
           
           'c_GM_CSF', 'c_IL_1_alpha', 'c_IL_5', 'c_IL_7', 'c_IL_12_IL_23p40', 'c_IL_15', 'c_IL_16',
           'c_IL_17A', 'c_TNF_beta', 'c_VEGF_A',
           
           'c_IL_1RA', 'c_IL_3', 'c_IL_9','c_IL_17A_F', 'c_IL_17B', 'c_IL_17C', 'c_IL_17D', 'c_TSLP',
           
           'c_IL_17AGenB', 'c_IL_21', 'c_IL_22', 'c_IL_23', 'c_IL_27', 'c_IL_31', 'c_MIP_3alpha',
           
           'c_bFGF', 'c_PlGF', 'c_Tie_2', 'c_VEGF_A_2', 'c_VEGF_C', 'c_VEGF_D', 'c_Fit_1',
           
           'c_CRP', 'c_ICAM_1', 'c_SAA', 'c_VCAM_1',
           
           'c_asyn', 'CSF_AB1_40', 'CSF_AB1_42', 'CSF_GFAP', 'CSF_nfl',
           'CSF_strem2', 'CSF_pdgfr_beta', 'CSF_pTau', 'CSF_tTau', 'CSF_pTau231', 'CSF_pTau217']

        
other = ['serum_albumin', 'csf_albumin', 'Qalb', 'p_Hb', 'CSF_hemolysis',]

fbm_final = ['p_Eotaxin', 'p_Eotaxin_3', 'p_IL_8', 'p_IL_8_2', 'p_IP_10', 'p_MCP_1', 'p_MCP_4', 
             'p_MDC', 'p_MIP_1alpha', 'p_MIP_1_beta', 'p_TARC',
             
             'p_INF_gamma', 'p_IL_1_beta', 'p_IL_2', 'p_IL_4', 'p_IL_6', 'p_IL_10', 
             'p_IL_12p70', 'p_IL_13', 'p_TNF_alpha',
             
             'p_GM_CSF', 'p_IL_1_alpha', 'p_IL_5', 'p_IL_7', 'p_IL_12_IL_23p40', 'p_IL_15', 
             'p_IL_16',  'p_IL_17A', 'p_TNF_beta', 'p_VEGF_A',

             'p_IL_1RA', 'p_IL_3', 'p_IL_9', 'p_IL_17A_F', 'p_IL_17B', 'p_IL_17C', 'p_IL_17D', 'p_TSLP',
             
             'p_IL_17AGenB', 'p_IL_21', 'p_IL_22', 'p_IL_23', 'p_IL_27', 'p_IL_31', 'p_MIP_3alpha',
             
             'p_bFGF', 'p_PlGF', 'p_Tie_2', 'p_VEGF_A_2', 'p_VEGF_C', 'p_VEGF_D', 'p_Fit_1',
             
             'p_CRP', 'p_ICAM_1', 'p_SAA', 'p_VCAM_1', 
             
             'p_asyn', 'Plasma_Ab40_avg', 'Plasma_Ab42_avg', 'Plasma_GFAP_avg', 'Plasma_NFL_avg',
             'Plasma_PDGFRbeta', 'pT181', 'pT231', 'Plasma_totaltau',
             
             'c_asyn', 'CSF_AB1_40', 'CSF_AB1_42', 'CSF_GFAP', 'CSF_nfl', 'CSF_strem2', 
             'CSF_pdgfr_beta', 'CSF_pTau', 'CSF_tTau', 'CSF_pTau231', 'CSF_pTau217',
             
             'c_Eotaxin', 'c_Eotaxin_3', 'c_IL_8', 'c_IL_8_2',  'c_IP_10', 'c_MCP_1', 'c_MCP_4', 
             'c_MDC', 'c_MIP_1alpha', 'c_MIP_1_beta', 'c_TARC',
           
             'c_INF_gamma',  'c_IL_1_beta', 'c_IL_2', 'c_IL_4', 'c_IL_6', 'c_IL_10', 'c_IL_12p70', 
             'c_IL_13', 'c_TNF_alpha',
           
             'c_GM_CSF', 'c_IL_1_alpha', 'c_IL_5', 'c_IL_7', 'c_IL_12_IL_23p40', 'c_IL_15', 'c_IL_16',
             'c_IL_17A', 'c_TNF_beta', 'c_VEGF_A',
           
             'c_IL_1RA', 'c_IL_3', 'c_IL_9','c_IL_17A_F', 'c_IL_17B', 'c_IL_17C', 'c_IL_17D', 'c_TSLP',
           
             'c_IL_17AGenB', 'c_IL_21', 'c_IL_22', 'c_IL_23', 'c_IL_27', 'c_IL_31', 'c_MIP_3alpha',
           
             'c_bFGF', 'c_PlGF', 'c_Tie_2', 'c_VEGF_A_2', 'c_VEGF_C', 'c_VEGF_D', 'c_Fit_1',
           
             'c_CRP', 'c_ICAM_1', 'c_SAA', 'c_VCAM_1',]

## drop participants with no data for plasma or csf

In [ ]:
datas = data.dropna(subset=['Plasma_GFAP_avg', 'Plasma_Ab42_avg', 'Plasma_PDGFRbeta', 'Plasma_totaltau'], axis=0)

datas = data.dropna(subset=['c_asyn', 'CSF_GFAP', 'CSF_tTau', 'CSF_pdgfr_beta', 'c_IL_1_alpha', 'CSF_pTau217'], axis=0)

print(datas.shape)
datas.head()

In [ ]:
datas[fbm_csf].isna().sum()

## remove params with too many imputed values

In [ ]:
plasma_exclude = ['p_IL_17AGenB', 'p_IL_8_2', 'p_IL_1_beta', 'p_IL_17A_F', 'p_IL_9', 'p_IL_4',
                 'p_IL_13', 'p_GM_CSF', 'p_IL_5']

csf_exclude = ['c_IL_12p70', 'c_IL_1_beta', 'c_MCP_4', 'c_IL_23', 'c_IL_17A_F', 'c_Tie_2',
               'c_IL_17C', 'c_IL_4', 'c_INF_gamma', 'c_IL_3', 'c_GM_CSF', 'c_IL_1_alpha',
               'c_VEGF_C', 'c_TNF_alpha', 'c_IL_1_alpha', 'c_IL_17AGenB', 'c_MIP_3alpha', 'c_IL_31']

In [ ]:
datas['TES_yes_no'].value_counts()

In [ ]:
for param in csf_exclude:
    print(param)
    
    sns.catplot(data=datas, y=param, kind="bar", x='TES_yes_no', ci=68)
    plt.show()


In [ ]:
#only exclude ones that have similar values across groups

plasma_exclude = ['p_IL_17AGenB', 'p_IL_1_beta', 'p_IL_9', 
                 'p_IL_13', 'p_GM_CSF', 'p_IL_5']

csf_exclude = ['c_IL_12p70', 'c_IL_1_beta', 'c_MCP_4', 'c_IL_17A_F', 'c_Tie_2',
               'c_IL_17C', 'c_IL_4', 'c_IL_3',
               'c_VEGF_C', 'c_TNF_alpha',  'c_IL_17AGenB', 'c_MIP_3alpha', ]

datas = datas.loc[:, ~data.columns.isin(csf_exclude)]
print(datas.shape)
datas.head()

In [ ]:
fbm_csf = ['c_Eotaxin', 'c_Eotaxin_3', 'c_IL_8', 'c_IL_8_2',  'c_IP_10', 'c_MCP_1',  
           'c_MDC', 'c_MIP_1alpha', 'c_MIP_1_beta', 'c_TARC',
           
           'c_INF_gamma',   'c_IL_2', 'c_IL_6', 'c_IL_10', 
           'c_IL_13', 
           
           'c_GM_CSF', 'c_IL_1_alpha', 'c_IL_5', 'c_IL_7', 'c_IL_12_IL_23p40', 'c_IL_15', 'c_IL_16',
           'c_IL_17A', 'c_TNF_beta', 'c_VEGF_A',
           
           'c_IL_1RA', 'c_IL_9', 'c_IL_17B','c_IL_17D', 'c_TSLP',
           
           'c_IL_21', 'c_IL_22', 'c_IL_23', 'c_IL_27', 'c_IL_31', 
           
           'c_bFGF', 'c_PlGF', 'c_VEGF_A_2', 'c_VEGF_D', 'c_Fit_1',
           
           'c_CRP', 'c_ICAM_1', 'c_SAA', 'c_VCAM_1',
           
           'c_asyn', 'CSF_AB1_40', 'CSF_AB1_42', 'CSF_GFAP', 'CSF_nfl',
           'CSF_strem2', 'CSF_pdgfr_beta', 'CSF_pTau', 'CSF_tTau', 'CSF_pTau231', 'CSF_pTau217']

## normalize

In [ ]:
normal = {}
i=0
for param in fbm_csf:
    print(param)
    
    sns.displot(datas, x=param, kind="kde", hue='TES_yes_no')
    plt.show()
    
    print(sm.stats.lilliefors(datas[param], dist='norm', pvalmethod='table'))
   
    #transform into normal dist
    normal[param] = stats.boxcox(datas[param])[0]
    

In [ ]:
normal_tranf = pd.DataFrame.from_dict(normal)
print(normal_tranf.shape)

normal_tranf

## corr

In [ ]:
fig, ax = plt.subplots(figsize=(45,30))
corr = datas[fbm_csf].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
ax = sns.heatmap(corr, annot=False, vmin=-1, vmax=1, center=0, cmap = 'coolwarm', 
                square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.xticks(rotation = 90)
plt.yticks(rotation = 0)
plt.show()

In [ ]:
def corr_sig(df=None):
    p_matrix = np.zeros(shape=(df.shape[1],df.shape[1]))
    for col in df.columns:
        for col2 in df.drop(col,axis=1).columns:
            _ , p = stats.spearmanr(df[col],df[col2])
            p_matrix[df.columns.to_list().index(col),df.columns.to_list().index(col2)] = p
    return p_matrix

def plot_cor_matrix(corr, mask=None):
    f, ax = plt.subplots(figsize=(45,30))
    sns.heatmap(corr, ax=ax,
                mask=mask,
                # cosmetics
                annot=True, vmin=-1, vmax=1, center=0,
                cmap='coolwarm', linewidths=1, linecolor='black', cbar_kws={'orientation': 'horizontal'})


# Plotting with significance filter  
corr = datas[fbm_csf].corr()
p_values = corr_sig(corr)                     # get p-Value
mask = np.invert(np.tril(p_values<0.01))    # mask - only get significant corr
plot_cor_matrix(corr,mask)  
plt.show()


## Cluster

In [ ]:
sns.color_palette("flare", as_cmap=True)

#lut = dict(zip(datas['TES_CTE_level_certainty'].unique(), "ybrc"))
#row_colors = datas['TES_CTE_level_certainty'].map(lut)
plt.figure(figsize=(30,45))

g = sns.clustermap(datas[fbm_csf], 
                 metric="euclidean", z_score=1, method="ward", 
               vmin=-2, vmax=2, center=0, cmap = 'coolwarm', 
               square=True, linewidths=.5, cbar_kws={"shrink": .5}, figsize=(45,45))
plt.show()

In [ ]:
# center and scale the data
scaler = StandardScaler()
#scaler = RobustScaler()

data_feat_scaled = scaler.fit_transform(datas[fbm_csf])

In [ ]:
data = data_feat_scaled

#pick cluster number based on silhouette coefficient
k_range = range(2,9)

base_scores = []
sil_scores = []
db_scores = []
ch_scores = []
mse_scores = []

for k in k_range:
    km_ss = KMeans(n_clusters=k, random_state=39)
    km_ss.fit(data)
    
    sil_scores.append(silhouette_score(data, km_ss.labels_))
    db_scores.append(davies_bouldin_score(data, km_ss.labels_))
    #ch_scores.append(calinski_harabasz_score(data, km_ss.labels_))
    mse_scores.append(km_ss.inertia_)
    
# plot the results
plt.plot(k_range, sil_scores)
plt.title('AUDIT-C Questions K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette Coefficient')
plt.show()

# plot the results
plt.plot(k_range, db_scores)
plt.title('AUDIT-C Questions K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Davies Bouldin Score')
plt.show()
    
# plot the results
plt.plot(k_range, mse_scores)
plt.title('AUDIT-C Questions K means clustering')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

In [ ]:
#viz dendrogram to find if three clusters makes sense
plt.figure(figsize=(30, 7))  
plt.title("Dendrogram 30day timepoint")  
plt.ylabel("Distance (dissimilarity)")
plt.xlabel("Participants")
dend = shc.dendrogram(shc.linkage(data_feat_scaled, method='ward'), 
                      distance_sort='ascending',
                      show_leaf_counts=True, leaf_font_size=8)

In [ ]:
#choose k=3 clusters and fit data
km_3 = KMeans(n_clusters=3,random_state=39)
km_3.fit(data_feat_scaled)
#add cluster info to df
datas['kmeans_cluster'] = ["cluster_" + str(label) for label in list(km_3.labels_)]

print(datas.shape)
print(datas['kmeans_cluster'].value_counts())
datas.head(1)

In [ ]:
plt.figure(figsize=(30, 7))  
datas.groupby('TES_yes_no')['kmeans_cluster'].value_counts()

sns.countplot(x='TES_yes_no', data=datas, hue='kmeans_cluster')

## viz

In [ ]:
datas = datas.replace({999: np.nan, 777: np.nan})

In [ ]:
'age_decade', 'age', 'racecat_combined', 'ethnusacat', 'ApoE_Result', 'e4_carrier', 
'gendertyp', 'eduyears', 'edulevel', 'marital', 'employ', 'insurance', 
'chneighb', 'chzipcode', 'milit',
        
       'checkin_bin', 'exposurebin', 'totyr_foot', 'chiiseas_pf',
       'chiiyrs_pf', 'chiiseas_pl', 'chiiyrs_pl', 'chiiseas_pg',
       'chiiyrs_pg', 'chiiseas_old', 'chiiyrs_old', 'partsport', 'pos_group',
    
    'headnecker', 'headneckcar',
       'headneckfall', 'headneckfight', 'nearblast', 'injuryko',
       'knjurydaze', 'concussfreqn', 'concusstotaln', 'blackoutblow',
    
       'TES_yes_no',
       'TES_CTE_level_certainty', 'TES_substan_RHI',
       'TES_cognitive_impair', 'TES_nb_dysreg', 'TES_progressive',
       'TES_not_accounted_for', 'TES_dementia', 'TES_SF_delayed_onset',
       'TES_SF_motor', 'TES_SF_motor_park', 'TES_SF_motor_other',
       'TES_SF_motor_mnd', 'TES_SF_psych_anxiety', 'TES_SF_psych_apathy',
       'TES_SF_psych_depress', 'TES_SF_psych_paranoia',
        
        'weight', 'height_bmi', 'bmi', 'bp_systolic', 'bp_diastolic', 'rFSRP',  
        'hypertd', 'hypochd',
        'diabd', 'diabtype', 'myoinf', 'myoinffreq', 'cardarr', 'cardarrfreq',
       'cad', 'angio', 'stent', 'carby', 'carbyvess', 'afib', 'elechrtd',
       'cardmyod', 'vulhrtd', 'pacemak', 'defibimp', 'conghfd', 'anginad',
       'conghd', 'cvdother', 'pervasd', 'stroketia', '
         
        'demd', 'parkd', 'epilepd', 'hdachehist', 'hdached', 
        'depd', 'anxd',
       'ptsdd', 'bipold', 'schizd', 'raged', 'psychd',
        
        'tmi', 'tmi_sp', 'tgec', 'tgec_sp',
       'tbri', 'tbri_sp', 'mocat', 'wratrstd', 'totfast', 'wordt',
       'colort', 'colwdt', 'intt', 'nabmazt', 'nsft', 'nsbt', 'nsfrawt',
       'nsftt', 'nsbrawt', 'nsbtt', 'sdmstd', 'attime', 'bttime',
       'bvmtrt', 'bvmdrt', 'craftvimmt', 'craftpimmt', 'craftvdelt',
       'craftpdelt', 'nlairt', 'nlsdt', 'nlldt', 'jolot', 'afnotott',
       'minttott', 
        
        'effort', 'bistot', 'aestot', 'baitot', 'baicat',
       'bditot', 'bdicat', 'bhstot', 'bhscat', 'bdhi_total', 'cnstot',
       'cnscat', 'cns_labile_laugh_score', 'cns_labile_tear_score',
       'pcltot', 'pclclusb', 'pclclusc', 'pclclusd', 'pclcluse',
       'ideaint', 'planning', 'tellrightp', 'childhoodp', 'warmp',
       'deafp', 'tellrightp_sp', 'childhoodp_sp', 'warmp_sp', 'deafp_sp',
       'npiqtot_sp', 'sangt', 'tangt', 'axixt', 'uclatot', 'eqstate',
       'BGLHA_Childhood_Total', 'BGLHA_Adolescence_Total',
       'BGLHA_Adulthood_Total', 'mdsupdrsp3_score', 
        
         'alcdrinkoft',
       'alcdrinkperday', 'alcsixdrinks', 'alcnotstop', 'alcfailtodo',
       'alcmorning', 'alcguilty', 'alcnotrem', 'alcinjure', 'alcconcern',
       'audit_tot', 'audit_4', 'illdrug___1', 'illdrug___2',
       'illdrug___3', 'illdrug___4', 'illdrug___5', 'illdrug___6',
       'illdrug___7', 'illdrug___0', 'presdrug___1', 'presdrug___2',
       'presdrug___3', 'presdrug___4', 'presdrug___0', 'drugpruse',
       'drugprname___1', 'drugprname___2', 'drugprname___3',
       'drugprname___4', 'drugprname___5', 'drugprname___6',
       'drugprname___7', 'drugprname___8', 'drugprname___9',
       'drugprname___10', 'drugprnameo', 'drugpragestart',
       'drugpragestop', 'drugprdur', 'drugadd', 'drugaddtold',
       'drugusebadjob', 'drivehigh', 'drugusetrouble', 'druguseproth',
       'drugusecut', 'drugusestop', 'performuse', 'injsteruse', 'hghuse',
       'epouse', 'hcguse', 'acthuse', 'stimuse', 'insulinuse',
       'toradoluse', 'cortisoneuse', 
    
    
        'cig100', 'cig30days', 'cigyrs',
       'cigavgpack', 'quitsmokeage', 'othtob100', 'othtob___1',
       'othtob___2', 'othtob___3', 'othtob___4', 'othtob___5',
       'othtob___6', 'othtob___999', 'othtob___777', 'othtobo',
       'othtob30days', 'othtobyears', 'quittobage', 'ends100',
       'ends30days', 'endsyears', 'quitendsage', 'nicrep100',
       'nicrep30days', 'nicrepyears', 'quitnicrepage', 

In [ ]:
datas[['alcdrinkoft',
       'alcdrinkperday', 'alcsixdrinks', 'alcnotstop', 'alcfailtodo',
       'alcmorning', 'alcguilty', 'alcnotrem', 'alcinjure', 'alcconcern',
       'audit_tot', 'audit_4', 'illdrug___1', 'illdrug___2',
       'illdrug___3', 'illdrug___4', 'illdrug___5', 'illdrug___6',
       'illdrug___7', 'illdrug___0', 'presdrug___1', 'presdrug___2',
       'presdrug___3', 'presdrug___4', 'presdrug___0', 'drugpruse',
       'drugprname___1', 'drugprname___2', 'drugprname___3',
       'drugprname___4', 'drugprname___5', 'drugprname___6',
       'drugprname___7', 'drugprname___8', 'drugprname___9',
       'drugprname___10', 'drugprnameo', 'drugpragestart',
       'drugpragestop', 'drugprdur', 'drugadd', 'drugaddtold',
       'drugusebadjob', 'drivehigh', 'drugusetrouble', 'druguseproth',
       'drugusecut', 'drugusestop', 'performuse', 'injsteruse', 'hghuse',
       'epouse', 'hcguse', 'acthuse', 'stimuse', 'insulinuse',
       'toradoluse', 'cortisoneuse',]]

In [ ]:
demogs = ['age_decade', 'age', 'racecat_combined', 'ethnusacat', 'e4_carrier', 
'gendertyp', 'eduyears', 'marital', 'employ', 'insurance', 
'chneighb', 'milit',]

exposure = ['checkin_bin', 'exposurebin', 'totyr_foot', 'chiiseas_pf',
       'chiiyrs_pf', 'chiiseas_pl', 'chiiyrs_pl', 'chiiseas_pg',
       'chiiyrs_pg', 'chiiseas_old', 'chiiyrs_old', 'partsport',
           'headnecker', 'headneckcar',
       'headneckfall', 'headneckfight', 'nearblast', 'injuryko',
       'knjurydaze', 'concussfreqn', 'concusstotaln', 'blackoutblow',]

diagnosis = ['TES_yes_no',
       'TES_CTE_level_certainty', 'TES_substan_RHI',
       'TES_cognitive_impair', 'TES_nb_dysreg', 
        'TES_dementia', 'TES_SF_delayed_onset',
       'TES_SF_motor', 'TES_SF_motor_park', 'TES_SF_motor_other',
       'TES_SF_motor_mnd', 'TES_SF_psych_anxiety', 'TES_SF_psych_apathy',
       'TES_SF_psych_depress', 'TES_SF_psych_paranoia',]

comorb = ['weight', 'height_bmi', 'bmi', 'bp_systolic', 'bp_diastolic', 'rFSRP',  
        'hypertd', 'hypochd',
        'diabd', 'diabtype', 'myoinf', 'myoinffreq', 'cardarr', 'cardarrfreq',
       'cad', 'angio', 'stent', 'carby', 'carbyvess', 'afib', 'elechrtd',
       'cardmyod', 'vulhrtd', 'pacemak', 'defibimp', 'conghfd', 'anginad',
       'conghd', 'cvdother', 'pervasd', 'stroketia', 
         ]

psychd = ['demd', 'parkd', 'epilepd', 'hdachehist', 'hdached', 
        'depd', 'anxd',
       'ptsdd', 'bipold', 'schizd', 'raged', 'psychd', ]

neurocog = ['tmi', 'tmi_sp', 'tgec', 'tgec_sp',
       'tbri', 'tbri_sp', 'mocat', 'wratrstd', 'totfast', 'wordt',
       'colort', 'colwdt', 'intt', 'nabmazt', 'nsft', 'nsbt', 'nsfrawt',
       'nsftt', 'nsbrawt', 'nsbtt', 'sdmstd', 'attime', 'bttime',
       'bvmtrt', 'bvmdrt', 'craftvimmt', 'craftpimmt', 'craftvdelt',
       'craftpdelt', 'nlairt', 'nlsdt', 'nlldt', 'jolot', 'afnotott',
       'minttott', ]  

neuropsych = ['bistot', 'aestot', 'baitot', 'baicat',
       'bditot', 'bdicat', 'bhstot', 'bhscat', 'bdhi_total', 'cnstot',
       'cnscat', 'cns_labile_laugh_score', 'cns_labile_tear_score',
       'pcltot', 'pclclusb', 'pclclusc', 'pclclusd', 'pclcluse',
       'ideaint', 'planning', 'tellrightp', 'childhoodp', 'warmp',
       'deafp', 'tellrightp_sp', 'childhoodp_sp', 'warmp_sp', 'deafp_sp',
       'npiqtot_sp', 'sangt', 'tangt', 'axixt', 'uclatot', 'eqstate',
       'BGLHA_Childhood_Total', 'BGLHA_Adolescence_Total',
       'BGLHA_Adulthood_Total', 'mdsupdrsp3_score', ]

subuse = ['cig100', 'othtob100',
          'alcdrinkoft',
       'alcdrinkperday', 'alcsixdrinks', 'alcnotstop', 'alcfailtodo',
       'alcmorning', 'alcguilty', 'alcnotrem', 'alcinjure', 'alcconcern',
       'audit_tot', 'audit_4', 'illdrug___1', 'illdrug___2',
       'illdrug___3', 'illdrug___4', 'illdrug___5', 'illdrug___6',
       'illdrug___7', 'illdrug___0', 'presdrug___1', 'presdrug___2',
       'presdrug___3', 'presdrug___4', 'presdrug___0', 'drugpruse',
       'drugprname___1', 'drugprname___2', 'drugprname___3',
       'drugprname___4', 'drugprname___5', 'drugprname___6',
       'drugprname___7', 'drugprname___8', 'drugprname___9',
       'drugprname___10',  'performuse', 'injsteruse', 'hghuse',
       'epouse', 'hcguse', 'acthuse', 'stimuse', 'insulinuse',
       'toradoluse', 'cortisoneuse', ]  


In [ ]:
list_o_params = [fbm_csf, fbm_serum, demogs, exposure, diagnosis, comorb, psychd, neurocog, neuropsych, subuse]

for param in list_o_params:
    print(param)
    
    data_z = pd.DataFrame(data=scipy.stats.zscore(datas[param], axis=0, nan_policy='omit'), columns=param)
    data_z['kmeans_cluster'] = datas['kmeans_cluster'].values

    df_melt = pd.melt(data_z, id_vars=['kmeans_cluster'], value_vars=param).dropna()

    groupby = df_melt.groupby(['kmeans_cluster', 'variable'])['value'].mean().reset_index()
    groupby = groupby.pivot('kmeans_cluster', 'variable', "value")
    groupby = groupby.reindex(columns=param)
    
    plt.figure(figsize=(40,5))
    ax = sns.heatmap(groupby, cmap="coolwarm", vmin=-1, vmax=1, square=True, 
                 linewidths=.5, 
                 cbar_kws={"shrink": .5},
                 )
    plt.yticks(rotation=0, fontsize="20", va="center")
    plt.show()